In [8]:
!pip install mxnet-cu90

  Using cached graphviz-0.8.4-py2.py3-none-any.whl (16 kB)
  Using cached numpy-1.16.6.zip (5.1 MB)
  Running setup.py clean for numpy
Failed to build numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
    Running setup.py install for numpy: started
    Running setup.py install for numpy: finished with status 'error'
  Rolling back uninstall of numpy
  Moving to c:\users\abdul rehman\anaconda3\lib\site-packages\numpy-1.18.5.dist-info\
   from C:\Users\Abdul Rehman\anaconda3\Lib\site-packages\~umpy-1.18.5.dist-info
  Moving to c:\users\abdul rehman\anaconda3\lib\site-packages\numpy\
   from C:\Users\Abdul Rehman\anaconda3\Lib\site-packages\~umpy
  Moving to c:\users\abdul rehman\anaconda3\scripts\f2py.exe
   from C:\Program Files\KMSpico\temp\pip-uninstall-93shiyid\f2py.exe


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Abdul Rehman\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Program Files\\KMSpico\\temp\\pip-install-yo9x5w3c\\numpy_85bcab74c14148208dadf268be87c67c\\setup.py'"'"'; __file__='"'"'C:\\Program Files\\KMSpico\\temp\\pip-install-yo9x5w3c\\numpy_85bcab74c14148208dadf268be87c67c\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Program Files\KMSpico\temp\pip-wheel-0csx_68c'
       cwd: C:\Program Files\KMSpico\temp\pip-install-yo9x5w3c\numpy_85bcab74c14148208dadf268be87c67c\
  Complete output (264 lines):
  Running from numpy source directory.
  C:\Program Files\KMSpico\temp\pip-install-yo9x5w3c\numpy_85bcab74c14148208dadf268be87c67c\numpy\distutils\misc_util.py:476: SyntaxWarning: "is" with a literal. Did you mean "=="?
    return is_stri

In [9]:
from mxnet import nd
from mxnet.gluon import nn
import d2lzh as d2l
import mxnet as mx
from mxnet.gluon import loss as gloss, nn
from mxnet import autograd, nd
import time
import matplotlib.pyplot as plt
import numpy as np
from mxnet.io import ImageRecordIter
import os, time, shutil
import random 
from shutil import copyfile
import cv2
from mxnet import gluon, image, init, nd
from mxnet import autograd as ag
from mxnet.gluon import nn
from mxnet.gluon.data.vision import transforms
from gluoncv.utils import makedirs
from gluoncv.model_zoo import get_model
from mxnet.gluon import data as gdata, loss as gloss, utils as gutils
import sys
from gluoncv.utils import viz

rec_path = os.path.abspath('images/')

ModuleNotFoundError: No module named 'mxnet'

In [14]:
model_name = 'mobilenetv3_small'
classes = 5
finetune_net = get_model(model_name, pretrained=True)

def conv_block(num_channels):
    blk = nn.HybridSequential() 
    blk.add(nn.Conv2D(num_channels,(1,1)),
            nn.BatchNorm(),
            nn.Activation('relu'),
            nn.GlobalAvgPool2D())
    return blk

class modifiedMobileNet(nn.HybridBlock):
  
    def __init__(self, **kwargs):
        super(modifiedMobileNet, self).__init__(**kwargs)
        finetune_net = get_model(model_name, pretrained=True)
        with self.name_scope():
          self.part1 = finetune_net.features[:3]
          self.part2 = finetune_net.features[3]
          self.part3 = finetune_net.features[4]
          self.part4 = finetune_net.features[5]
          self.part5 = finetune_net.features[6:21]
          self.part6 = nn.Dense(classes)
         # self.part6 = nn.Conv2D(classes, kernel_size=(1, 1))
          self.flatten = nn.Flatten()
          self.dropout = nn.Dropout(0.5)
          self.conv1 = conv_block(64)
          self.conv2 = conv_block(64)
          self.conv3 = conv_block(64)
          self.conv4 = conv_block(64)
          self.conv5 = conv_block(64)
          self.w = conv_block(5)
     

    def forward(self,x):
            w = self.w(x)
            w = self.flatten(w)
       #     print(w.shape)
            x = self.part1(x)
            x1 = self.conv1(x)
       #     print(x1.shape)
            x = self.part2(x)
            x2 = self.conv2(x)
       #     print(x2.shape)
            x = self.part3(x)
            x3 = self.conv3(x)
      #      print(x3.shape)
            x = self.part4(x)
            x4 = self.conv4(x)
      #      print(x4.shape)
            x = self.part5(x)
            x5 = self.conv5(x)
      #      print(x5.shape)
            x = w[0]*x1+w[1]*x2+w[2]*x3+w[3]*x4+w[4]*x5
          #  x = nd.concat(x1, x2, x3, x4, x5, dim = 1)
            x = self.dropout(x)
            x = self.part6(x)
            return x

net1 =  modifiedMobileNet()
net1.load_parameters('k_fold_cross_validation/models/net_3.params')
net1.collect_params().reset_ctx(ctx)

In [15]:
model_name = 'mobilenetv3_small'
classes = 5
loss = gloss.SoftmaxCrossEntropyLoss()    

finetune_net = get_model(model_name, pretrained=True)
output = nn.HybridSequential()
output.add(nn.Conv2D(classes, (1,1)))
output.add(nn.Flatten())
with finetune_net.name_scope():
    finetune_net.output = output
finetune_net.output.initialize(init.Xavier(), ctx = ctx)

finetune_net.load_parameters('dataAll_5_class/models/mobilenetv3_small2.params')
finetune_net.collect_params().reset_ctx(ctx)

In [16]:
ctx

[gpu(0)]

In [123]:
test_data = ImageRecordIter(
    path_imgrec = os.path.join(rec_path, 'Test.rec'),
    path_imgidx = os.path.join(rec_path, 'Test.idx'),
    resize = 299,
    data_shape  = (3, 299, 299),
    batch_size  = 20,  
    preprocess_threads = 12,
    shuffle = False,
)


class DataIterLoader():
  def __init__(self, data_iter):
    self.data_iter = data_iter

  def __iter__(self):
    self.data_iter.reset()
    return self

  def __next__(self):
    batch = self.data_iter.__next__()
    assert len(batch.data) == len(batch.label) == 1
    data = batch.data[0]
    label = batch.label[0]
    return data, label

test_iter = DataIterLoader(test_data)

In [17]:
final_test_acc_noise1 = evaluate_accuracy(test_iter, finetune_net, ctx)
print('mobilenetv3_small:', final_test_acc_noise1)

mobilenetv3_small: 0.99


In [18]:
final_test_acc_noise1 = evaluate_accuracy(test_iter, net1, ctx)
print('modified mobilenetv3_small:', final_test_acc_noise1)

modified mobilenetv3_small: 0.995


In [13]:
def try_all_gpus():  
    ctxes = []
    try:
        for i in range(16):  
            ctx = mx.gpu(i)
            _ = nd.array([0], ctx=ctx)
            ctxes.append(ctx)
    except mx.base.MXNetError:
        pass
    if not ctxes:
        ctxes = [mx.cpu()]
    return ctxes

ctx = try_all_gpus()

def _get_batch(batch, ctx):
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])
    
def evaluate_accuracy(data_iter, net, ctx=[mx.cpu()]):
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n = nd.array([0]), 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            y = y.astype('float32')
            acc_sum += (net(X).argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n




In [43]:
def robustness(n,m):
    #n  =  nd.random.normal(average,1)
    #m  =  nd.random.normal(average,1)
    print(n, m)

    class DataIterLoader_test():

        def __init__(self, data_iter):
            self.data_iter = data_iter
            self.n =  n
            self.m = m

        def __iter__(self):
            self.data_iter.reset()
            return self

        def __next__(self):
            batch = self.data_iter.__next__()
            assert len(batch.data) == len(batch.label) == 1
            mean_value  = nd.mean(nd.flatten(batch.data[0])).asscalar()
            #data = self.n * nd.random.normal(0,1,shape=batch.data[0].shape) * batch.data[0] + self.m  * nd.random.normal(0,1,shape=batch.data[0].shape)
            data = self.n * batch.data[0] + self.m  * nd.random.normal(0,1,shape=batch.data[0].shape)
            #data= (data  - nd.min(data))/(nd.max(data) -  nd.min(data)) *  255
            data = nd.rint(data.clip(0,255))
            label = batch.label[0]
            return data, label

    test_iter_noise = DataIterLoader_test(test_data)
    
    for data, label in  test_iter_noise:
        print(data)
        break

    final_test_acc_noise1 = evaluate_accuracy(test_iter_noise, finetune_net, ctx)
    print('original mobilenetv3_small:', final_test_acc_noise1)

    final_test_acc_noise1 = evaluate_accuracy(test_iter_noise, net1, ctx)
    print('modified  mobilenetv3_small:', final_test_acc_noise1)

In [40]:
robustness(1.1,2)

1.1 2

[[[[ 16.  12.  14. ...  12.  14.   9.]
   [ 15.  13.  13. ...  11.  14.   6.]
   [ 17.  11.  13. ...  14.  11.   2.]
   ...
   [ 11.  11.  10. ...  13.  11.   4.]
   [ 10.  10.  10. ...  13.  11.   6.]
   [ 10.  14.  10. ...  10.  11.   3.]]

  [[ 14.  17.  18. ...  13.  10.   6.]
   [ 17.  13.  16. ...  11.  11.   6.]
   [ 20.  15.  10. ...  11.  11.   7.]
   ...
   [ 11.  10.  12. ...  11.   9.   5.]
   [ 11.   8.  11. ...  14.  10.   5.]
   [ 10.  11.  14. ...  12.   8.   4.]]

  [[ 14.  13.  17. ...  12.  10.   6.]
   [ 13.  13.  15. ...  13.  11.  11.]
   [ 15.  14.  12. ...  13.  11.   5.]
   ...
   [ 12.   8.  11. ...   9.   8.   6.]
   [ 11.   9.   9. ...  11.  13.   9.]
   [ 12.  12.   7. ...  13.  10.   3.]]]


 [[[  3.   3.   3. ...   0.   0.   0.]
   [  2. 133. 255. ...   1.   4.   0.]
   [  0. 131. 255. ...   0.   0.   1.]
   ...
   [  0.   0.   0. ...   0.   1.   1.]
   [  1.   0.   1. ...   1.   0.   0.]
   [  2.   0.   0. ...   0.   0.   0.]]

  [[  0.   0.   0. 

In [41]:
robustness(1.2,2)

1.2 2

[[[[ 14.  13.  18. ...  13.  13.   9.]
   [ 16.  17.  17. ...  14.  11.   7.]
   [ 17.  15.  13. ...  15.  12.   3.]
   ...
   [ 12.  11.  14. ...  14.  14.  11.]
   [ 12.  12.  13. ...  14.  14.   7.]
   [ 10.  11.  12. ...  16.  14.   8.]]

  [[ 16.  17.  19. ...  13.  13.   4.]
   [ 17.  15.  18. ...  14.  11.   7.]
   [ 18.  14.  16. ...  17.  16.   5.]
   ...
   [  8.  14.  13. ...  15.  14.   8.]
   [ 14.  12.  11. ...  14.  14.   6.]
   [ 13.  14.  11. ...  12.  12.   6.]]

  [[ 14.  15.  19. ...  15.  12.   7.]
   [ 11.  14.  15. ...  18.  14.   9.]
   [ 17.  16.  11. ...  13.  10.   5.]
   ...
   [ 14.  13.  13. ...  16.  14.   5.]
   [ 11.  12.  12. ...  13.  10.   3.]
   [ 12.  12.  14. ...  13.  15.   9.]]]


 [[[  3.   0.   0. ...   1.   0.   2.]
   [  4. 147. 255. ...   2.   0.   7.]
   [  0. 141. 255. ...   4.   0.   0.]
   ...
   [  1.   1.   6. ...   1.   0.   2.]
   [  6.   0.   0. ...   0.   0.   0.]
   [  0.   0.   1. ...   2.   0.   1.]]

  [[  5.   4.   1. 

In [42]:
robustness(1.3,2)

1.3 2

[[[[ 15.  13.  16. ...  18.  14.   8.]
   [ 18.  16.  17. ...  16.  16.   8.]
   [ 18.  16.  16. ...  18.  17.   5.]
   ...
   [ 13.  11.  15. ...  17.  14.   6.]
   [ 16.  15.  10. ...  14.  14.   5.]
   [ 15.  12.  15. ...  16.  11.   8.]]

  [[ 13.  14.  15. ...  19.  13.   5.]
   [ 15.  16.  20. ...  14.  14.   8.]
   [ 16.  17.  18. ...  17.  17.   7.]
   ...
   [ 11.  13.  13. ...  15.  11.   7.]
   [ 14.  12.  11. ...  17.  18.   6.]
   [ 14.  15.  14. ...  14.  16.   6.]]

  [[ 18.  15.  18. ...  15.  11.   6.]
   [ 17.  15.  18. ...  16.  12.  10.]
   [ 17.  15.  18. ...  14.  15.   5.]
   ...
   [ 10.  13.  14. ...  14.  13.   9.]
   [ 13.  13.  15. ...  19.  13.   8.]
   [ 12.  20.  14. ...  16.  14.   5.]]]


 [[[  3.   3.   0. ...   0.   0.   0.]
   [  1. 158. 255. ...   0.   1.   0.]
   [  2. 157. 255. ...   2.   0.   1.]
   ...
   [  0.   2.   2. ...   1.   1.   0.]
   [  0.   0.   0. ...   0.   0.   1.]
   [  2.   0.   1. ...   0.   0.   0.]]

  [[  2.   3.   0. 

In [44]:
robustness(1.4,2)

1.4 2

[[[[ 19.  20.  16. ...  15.  17.   6.]
   [ 16.  17.  20. ...  13.  15.   9.]
   [ 17.  15.  17. ...  15.  17.   5.]
   ...
   [ 10.  16.  14. ...  15.  13.   8.]
   [ 14.  14.  14. ...  12.  14.   7.]
   [ 12.  15.  15. ...  15.  13.   8.]]

  [[ 19.  19.  17. ...  17.  15.   6.]
   [ 17.  16.  17. ...  14.  17.   9.]
   [ 16.  16.  17. ...  17.  18.   4.]
   ...
   [ 18.  11.  10. ...  18.  19.   9.]
   [ 13.  11.  12. ...  15.  13.   2.]
   [ 17.  16.  14. ...  14.  15.  10.]]

  [[ 18.  16.  17. ...  19.  17.  10.]
   [ 20.  20.  20. ...  17.  17.  11.]
   [ 17.  18.  14. ...  15.  19.   8.]
   ...
   [ 15.  17.  15. ...  15.  16.   6.]
   [ 15.  13.  13. ...  13.  14.   5.]
   [ 12.  12.  16. ...  16.  13.   4.]]]


 [[[  1.   6.   0. ...   0.   1.   0.]
   [  0. 167. 255. ...   1.   1.   3.]
   [  3. 167. 255. ...   0.   2.   0.]
   ...
   [  0.   0.   2. ...   0.   0.   1.]
   [  0.   1.   0. ...   0.   3.   0.]
   [  0.   5.   0. ...   0.   0.   0.]]

  [[  0.   0.   2. 

In [45]:
robustness(1.5,2)

1.5 2

[[[[ 19.  17.  21. ...  19.  15.   6.]
   [ 22.  17.  20. ...  17.  17.  12.]
   [ 23.  16.  17. ...  16.  21.   6.]
   ...
   [ 15.  16.  17. ...  16.  18.   7.]
   [ 15.  12.  15. ...  18.  16.   7.]
   [ 17.  16.  12. ...  15.  15.   8.]]

  [[ 22.  20.  26. ...  18.  18.  10.]
   [ 18.  22.  16. ...  15.  17.   8.]
   [ 23.  20.  19. ...  16.  18.   9.]
   ...
   [ 15.  13.  17. ...  14.  16.   7.]
   [ 14.  13.  12. ...  19.  15.   9.]
   [ 14.  17.  12. ...  14.  15.   9.]]

  [[ 15.  15.  22. ...  19.  17.   8.]
   [ 21.  18.  19. ...  18.  17.   9.]
   [ 22.  18.  15. ...  18.  19.   5.]
   ...
   [ 15.  16.  16. ...  16.  16.  12.]
   [ 12.  16.  15. ...  17.  16.   3.]
   [ 18.  16.  14. ...  17.  16.  11.]]]


 [[[  0.   2.   0. ...   0.   0.   1.]
   [  2. 178. 255. ...   1.   2.   0.]
   [  5. 182. 255. ...   4.   0.   0.]
   ...
   [  2.   1.   0. ...   1.   1.   3.]
   [  0.   0.   3. ...   2.   0.   0.]
   [  0.   1.   0. ...   0.   0.   1.]]

  [[  4.   7.   0. 

In [47]:
robustness(0.9,2)

0.9 2

[[[[ 14.   9.  13. ...  12.  11.   1.]
   [ 12.   8.  16. ...  14.  14.   4.]
   [ 17.  12.  15. ...   6.  12.   5.]
   ...
   [  9.  11.   6. ...   9.  12.   3.]
   [ 11.  10.  12. ...   9.  13.   2.]
   [  8.   8.   9. ...   3.  10.   8.]]

  [[ 12.   9.  12. ...   9.  12.   7.]
   [  9.   9.  12. ...  11.   8.   3.]
   [ 10.   9.  12. ...   8.  10.   6.]
   ...
   [ 10.  11.  10. ...   8.  10.   1.]
   [  8.  14.  10. ...  15.  12.   5.]
   [  8.   9.   8. ...  10.   8.   6.]]

  [[ 12.  12.  11. ...  12.  11.   6.]
   [ 13.  11.  12. ...  10.  10.   9.]
   [ 11.  12.  10. ...  11.   9.   7.]
   ...
   [  8.  11.   8. ...   9.  12.   7.]
   [  8.   9.  10. ...   8.   4.   5.]
   [  9.  11.   7. ...  11.  10.   7.]]]


 [[[  0.   4.   0. ...   0.   0.   2.]
   [  0. 107. 228. ...   1.   0.   2.]
   [  0. 107. 229. ...   0.   4.   5.]
   ...
   [  2.   0.   3. ...   2.   3.   3.]
   [  2.   1.   1. ...   2.   3.   0.]
   [  0.   2.   0. ...   0.   1.   2.]]

  [[  4.   5.   0. 

In [48]:
robustness(0.8,2)

0.8 2

[[[[ 14.   8.  10. ...   7.   8.  10.]
   [  7.  11.  11. ...   9.   7.   6.]
   [  8.  10.  11. ...   8.  10.   5.]
   ...
   [ 10.   6.  10. ...   4.   8.   5.]
   [  4.   7.  11. ...   9.   9.   5.]
   [ 10.  10.   9. ...   6.   9.   6.]]

  [[  8.  10.  10. ...   8.   8.   4.]
   [  6.  10.  10. ...  12.   8.   7.]
   [ 10.  10.   6. ...  10.   8.   4.]
   ...
   [ 12.   8.  10. ...  12.   9.   3.]
   [ 10.   9.  10. ...   8.   6.   2.]
   [  7.   5.   6. ...   9.  11.   4.]]

  [[ 10.  10.  13. ...   9.   7.   6.]
   [  9.   9.  10. ...   4.  13.   0.]
   [  9.  12.  12. ...  10.  13.   4.]
   ...
   [  8.   8.   5. ...   7.  10.   3.]
   [ 10.   7.   5. ...  13.   8.   2.]
   [  2.   8.   8. ...   9.   8.   7.]]]


 [[[  1.   1.   2. ...   0.   0.   0.]
   [  0.  94. 203. ...   0.   0.   0.]
   [  3.  96. 203. ...   0.   0.   1.]
   ...
   [  0.   0.   2. ...   1.   1.   0.]
   [  0.   0.   0. ...   1.   0.   4.]
   [  0.   0.   0. ...   0.   3.   1.]]

  [[  2.   2.   0. 

In [49]:
robustness(0.7,2)

0.7 2

[[[[  9.   7.  12. ...   7.   7.   5.]
   [ 10.   8.   9. ...   6.   6.   8.]
   [  8.   7.   7. ...  11.   6.   2.]
   ...
   [  8.   7.   5. ...   8.   3.   5.]
   [  7.   3.   6. ...   9.  11.   5.]
   [  7.   8.  10. ...  11.  11.   5.]]

  [[  9.   8.   8. ...  11.   7.   3.]
   [  6.   7.  10. ...   9.   9.   1.]
   [  6.  10.  10. ...   8.   9.   2.]
   ...
   [  6.   7.   3. ...  10.   9.   0.]
   [  5.   7.  10. ...   4.   7.   5.]
   [  5.   3.   6. ...   8.   9.   0.]]

  [[  8.   9.  10. ...   9.   8.   4.]
   [  9.   8.   8. ...   8.   7.   4.]
   [ 10.  11.  12. ...   8.   4.   5.]
   ...
   [ 10.   8.   5. ...   8.   7.   4.]
   [  7.  11.   6. ...   7.   8.   3.]
   [  3.   8.   4. ...   9.   6.   3.]]]


 [[[  8.   0.   0. ...   0.   0.   1.]
   [  0.  85. 178. ...   2.   0.   2.]
   [  1.  85. 178. ...   0.   0.   0.]
   ...
   [  1.   0.   0. ...   0.   0.   3.]
   [  0.   0.   3. ...   0.   2.   0.]
   [  0.   0.   0. ...   0.   0.   0.]]

  [[  0.   2.   0. 

In [50]:
robustness(0.6,2)

0.6 2

[[[[  9.  10.   8. ...   5.   7.   6.]
   [ 10.   9.  10. ...  10.   7.   5.]
   [  9.   7.   4. ...   8.  11.   3.]
   ...
   [  8.   7.   8. ...   4.   8.   4.]
   [  6.   7.   7. ...   6.   2.   4.]
   [  4.   3.   5. ...   9.   8.   3.]]

  [[  9.   7.   9. ...   7.   9.   3.]
   [  8.  10.   7. ...   7.   5.   2.]
   [ 10.   4.   8. ...   7.   6.   4.]
   ...
   [  5.   3.   4. ...   8.   7.   3.]
   [  7.   6.   8. ...   8.   7.   4.]
   [  5.   7.   3. ...   6.   5.   4.]]

  [[  5.   2.   7. ...   7.   8.   5.]
   [  4.  10.   7. ...   9.   7.   2.]
   [  8.   8.   6. ...   5.   9.   1.]
   ...
   [  7.   7.  10. ...   9.   9.   2.]
   [  3.   6.   9. ...   9.   9.   8.]
   [  5.   5.  11. ...   9.   7.   7.]]]


 [[[  4.   0.   0. ...   3.   1.   0.]
   [  0.  73. 154. ...   0.   4.   4.]
   [  0.  70. 149. ...   2.   0.   1.]
   ...
   [  0.   0.   1. ...   1.   0.   0.]
   [  0.   0.   0. ...   0.   3.   1.]
   [  1.   0.   0. ...   1.   4.   0.]]

  [[  1.   1.   1. 

In [51]:
robustness(0.5,2)

0.5 2

[[[[  6.   5.  10. ...   5.   3.   7.]
   [  5.   5.   4. ...   9.   5.   4.]
   [  7.   7.   7. ...   6.   8.   1.]
   ...
   [  5.   5.   3. ...   6.   6.   4.]
   [  3.   5.   6. ...   7.   8.   0.]
   [  7.   6.   5. ...   1.   3.   4.]]

  [[  4.   4.   9. ...   7.   6.   2.]
   [  5.   4.   5. ...   7.   7.   5.]
   [  8.   6.   6. ...   7.   8.   4.]
   ...
   [  2.   3.   6. ...   5.   5.   5.]
   [  4.   6.   7. ...   8.   8.   0.]
   [  4.   2.   8. ...   4.   3.   0.]]

  [[  2.   6.   7. ...   9.   6.   0.]
   [  6.   6.   7. ...   2.   4.   5.]
   [  8.   4.  10. ...   5.   8.   0.]
   ...
   [  2.   9.   5. ...   3.   7.   1.]
   [ 12.   4.   6. ...   3.   3.   3.]
   [  3.   6.   6. ...   5.   5.   1.]]]


 [[[  0.   0.   0. ...   0.   2.   0.]
   [  0.  59. 126. ...   0.   3.   0.]
   [  0.  60. 126. ...   4.   3.   2.]
   ...
   [  0.   2.   0. ...   0.   1.   0.]
   [  2.   2.   0. ...   0.   0.   0.]
   [  0.   0.   2. ...   2.   6.   3.]]

  [[  1.   2.   0. 